In [ ]:
import os, sys
import torch
import numpy as np
import csv
import cv2
import matplotlib.pyplot as plt

import time

import multiprocessing as mp

In [ ]:
with open('./data/OU_ISIR/ID_list.csv', 'r') as ID_list:
    lines = ID_list.readlines()
    
cooked = csv.reader(lines)
train_IDs = []
test_IDs = []
for record in cooked:
    if record[0] == 'Training subject ID' or record[0] == '':
        continue
    train_IDs.append(int(record[0]))
    test_IDs.append(int(record[1]))

In [ ]:
def zero_pad(num, digit=3):
    n_str = str(num)
    while len(n_str) < digit:
        n_str = "0" + n_str
    return n_str

In [ ]:
def crop_pedestrian(img, target_size):
    
    if img.sum() <= 5000:
        return None
    
    y = img.sum(dim=1)    
    y_top = (y!=0).argmax(dim=0)
    y_btm = (y!=0).cumsum(dim=0).argmax(dim=0)
    
    img = img[ y_top : y_btm + 1, :]
    
    asp_ratio = img.size(1) / img.size(0)    
    width = int(target_size * asp_ratio)
    
    img = cv2.resize(img.numpy(), (width, target_size), interpolation=cv2.INTER_AREA)
    img = torch.from_numpy(img)
    
    x_thres = img.sum() / 2
    sum_col = img.sum(dim=0).cumsum(dim=0)
    
    x_center = -1
    for i in range(len(sum_col)):        
        if sum_col[i] > x_thres:
            x_center = i
            break
    
    x_left = x_center - (target_size // 2)
    x_right = x_center + (target_size // 2)
    
    if ( x_left <= 0 ) or ( x_right >= 0):        
        x_left = x_left + (target_size // 2)
        x_right = x_right + (target_size // 2)
        
        tmp_img = torch.zeros(img.size(0), (target_size // 2), dtype=torch.uint8)
        img = torch.cat((tmp_img, img, tmp_img), dim=1)
    
    img = img[: , x_left : x_right]
    
    return img

In [ ]:
def crop_seq(path):
    for img_file in os.listdir( path ):
        
        img = cv2.imread( os.path.join( path, img_file))
        img = crop_pedestrian(torch.from_numpy(img)[:,:,0], 64)
        
        if img is not None:
            cv2.imwrite( os.path.join( path.replace('raw', 'processed'), img_file), img.numpy())

In [ ]:
root_path = './data/OU_ISIR/raw/'

os.makedirs( root_path.replace('raw','processed'), exist_ok=True)

start_time = time.time()

pool = mp.Pool(processes=6)

for seq_path in os.listdir(root_path):
    
    tmp_path = os.path.join(root_path, seq_path)
    os.makedirs(tmp_path.replace('raw', 'processed'), exist_ok=True)
    
    for idx in range(1000):
        file_path = os.path.join(tmp_path, zero_pad(train_IDs[idx + 4000], 5))
        
        if not os.path.exists(file_path):
            continue
            
        if not os.path.exists(file_path.replace('raw', 'processed')):
            os.mkdir(file_path.replace('raw','processed'))
        else:
            continue
        
        crop_seq(file_path)
        
    print(seq_path, ' is finished ')
        
pool.close()
pool.join()

print(' [ Start_time ] : ', start_time)
print(' [ Elapsed time ] :', time.time() - start_time)

pool = mp.Pool(processes=6)

for seq_path in os.listdir(root_path):
    
    tmp_path = os.path.join(root_path, seq_path)
    os.makedirs(tmp_path.replace('raw', 'processed'), exist_ok=True)
    
    for idx in range(1000):
        file_path = os.path.join(tmp_path, zero_pad(train_IDs[idx + 5000], 5))
        
        if not os.path.exists(file_path):
            continue
            
        if not os.path.exists(file_path.replace('raw', 'processed')):
            os.mkdir(file_path.replace('raw','processed'))
        else:
            continue
        
        crop_seq(file_path)
        
    print(seq_path, ' is finished ')
        
pool.close()
pool.join()

print(' [ Start_time ] : ', start_time)
print(' [ Elapsed time ] :', time.time() - start_time)